In [ ]:
# Scraping pagina 1 de vivino

import requests
from bs4 import BeautifulSoup
import re 

url = "https://www.vivino.com/es/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1NTBQS660TS1WS7Z1DQ1SKwDKpqfZliUWZaaWJOao5Rel2KakFier5SdV2hYUZSanqpUXR8faGloYqBUBaRMDUwBHtxqD"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0 Safari/537.36",
}

resp = requests.get(url, headers=headers)
resp.raise_for_status()  # por si hay error de red
soup = BeautifulSoup(resp.text, "html.parser")

# 1) Nombre del vino
name_elems = soup.select('div[class^="wineInfoVintage__truncate"]')
names = [el.get_text(strip=True) for el in name_elems]

# 1b) Añada / Vintage
vintage_elems = soup.select('div[class^="wineInfoVintage__vintage"]')
vintages = [el.get_text(strip=True) for el in vintage_elems]

# 2) Rating
rating_elems = soup.select('div[class^="vivinoRating__averageValue"]')
ratings_raw = [el.get_text(strip=True) for el in rating_elems]

def limpiar_rating(texto):
    if not texto:
        return None
    return float(texto.replace(",", "."))

ratings = [limpiar_rating(r) for r in ratings_raw]

# 3) Precio
price_elems = soup.select('div[class^="addToCartButton__price"]')
prices_raw = [el.get_text(strip=True) for el in price_elems]

def limpiar_precio(texto):
    # Ejemplos: 'EUR\xa03,95', 'EUR 12,50', '19,90 €'
    texto = texto.replace("\xa0", " ")
    match = re.search(r"(\d+[.,]\d+|\d+)", texto)
    if match:
        numero = match.group(1).replace(",", ".")
        return float(numero)
    return None

prices = [limpiar_precio(p) for p in prices_raw]


# Juntar todo (coger el mínimo común para evitar desajustes)
wines = []
for n, v, r, p in zip(names, vintages, ratings, prices):
    titulo_completo = f"{n} ({v})"  # nombre + añada
    wines.append({
        "titulo": titulo_completo,
        "nombre": n,
        "vintage": v,
        "rating": r,
        "precio": p
    })

if not wines:
    print("No se ha encontrado ningún vino")
else:
    for w in wines:
        print(w)


In [1]:

#OTRA FORMA DE HACERLO CON APIS:

import requests
import time

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/123.0 Safari/537.36"
    )
}

# Parámetros base sacados del href de tus botones
PARAMS_BASE = {
    "country_code": "ES",          # código país principal (España)
    "country_codes[]": "es",       # vinos de España
    "currency_code": "EUR",
    "min_rating": 0,
    "order_by": "price",
    "order": "desc",
    "price_range_min": 0,
    "price_range_max": 500,        # "todos" los precios en la búsqueda que hiciste
    "discount_prices": "false",
    "wine_style_ids[]": "180",     # estilo: Ribera del Duero Tinto (España)
    "region_ids[]": "405",         # región: Ribera del Duero
    "wine_type_ids[]": "1",        # tipo de vino: tinto
    "wsa_year": "null",
}


API_URL = "https://www.vivino.com/api/explore/explore"

all_wines = []
LAST_PAGE = 92

for page in range(1, LAST_PAGE + 1):
    params = PARAMS_BASE.copy()
    params["page"] = page

    print(f"Scrapeando página {page}...")

    r = requests.get(API_URL, params=params, headers=HEADERS)
    r.raise_for_status()
    data = r.json()

    matches = data["explore_vintage"]["matches"]
    if not matches:
        print(f"Página {page} sin resultados, paro aquí.")
        break

    for t in matches:
        vintage = t["vintage"]
        wine = vintage["wine"]

        wine_name = wine["name"]
        year = vintage["year"]
        rating = vintage["statistics"]["ratings_average"]

        price_info = t.get("price") or {}
        price = price_info.get("amount")   # ya es número, no hace falta regex

        titulo_completo = f"{wine_name} ({year})"

        all_wines.append({
            "titulo": titulo_completo,
            "nombre": wine_name,
            "vintage": year,
            "rating": rating,
            "precio": price,
            "pagina": page,
        })

    print(f"  -> {len(matches)} vinos en página {page}")
    time.sleep(0.5)   # pequeña pausa por respeto al servidor

print(f"\nTotal de vinos recogidos: {len(all_wines)}")

# Igual que antes: ver primeros
for w in all_wines[:10]:
    print(w)


Scrapeando página 1...
  -> 24 vinos en página 1
Scrapeando página 2...
  -> 24 vinos en página 2
Scrapeando página 3...
  -> 24 vinos en página 3
Scrapeando página 4...
  -> 24 vinos en página 4
Scrapeando página 5...
  -> 24 vinos en página 5
Scrapeando página 6...
  -> 24 vinos en página 6
Scrapeando página 7...
  -> 24 vinos en página 7
Scrapeando página 8...
  -> 24 vinos en página 8
Scrapeando página 9...
  -> 24 vinos en página 9
Scrapeando página 10...
  -> 24 vinos en página 10
Scrapeando página 11...
  -> 24 vinos en página 11
Scrapeando página 12...
  -> 24 vinos en página 12
Scrapeando página 13...
  -> 24 vinos en página 13
Scrapeando página 14...
  -> 24 vinos en página 14
Scrapeando página 15...
  -> 24 vinos en página 15
Scrapeando página 16...
  -> 24 vinos en página 16
Scrapeando página 17...
  -> 24 vinos en página 17
Scrapeando página 18...
  -> 24 vinos en página 18
Scrapeando página 19...
  -> 24 vinos en página 19
Scrapeando página 20...
  -> 24 vinos en página 2

In [2]:
#Generar DF y pasarlo a excel

import pandas as pd

df_vinos = pd.DataFrame(all_wines)
print(df_vinos.head())

df_vinos.to_excel("vinos_vivino.xlsx", index=False)


                        titulo                nombre vintage  rating   precio  \
0                Pingus (2004)                Pingus    2004     4.7  4001.25   
1                Pingus (2015)                Pingus    2015     4.7  2390.00   
2                Pingus (2018)                Pingus    2018     4.6  2240.00   
3  Unico (Gran Reserva) (1941)  Unico (Gran Reserva)    1941     4.6  2178.00   
4  Unico (Gran Reserva) (1959)  Unico (Gran Reserva)    1959     4.6  1916.40   

   pagina  
0       1  
1       1  
2       1  
3       1  
4       1  


In [ ]:
import requests
import time
import pandas as pd

URL = "https://www.vivino.com/api/explore/explore"  # o la URL que estés usando

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/123.0 Safari/537.36"
    )
}

# Parámetros base (los tuyos, sin la página)
PARAMS_BASE = {
    "currency_code": "EUR",
    "min_rating": 0,
    "order_by": "price",
    "order": "desc",
    "price_range_min": 0,
    "price_range_max": 500,
    "discount_prices": "false",
    "wine_style_ids[]": 180,   # Ribera del Duero tinto, por tu ejemplo
    "country_codes[]": "es",
    "wine_type_ids[]": 1,      # vino tinto
    "region_ids[]": 405        # Ribera del Duero
    # si usas el parámetro "e", lo añades aquí también
}

NUM_PAGES = 100

all_wines = []

for page in range(1, NUM_PAGES + 1):
    print(f"Scrapeando página {page}...")

    params = PARAMS_BASE.copy()
    params["page"] = page

    response = requests.get(URL, headers=HEADERS, params=params)
    response.raise_for_status()

    data = response.json()

    explore_data = data.get("explore_vintage") or data.get("exploreV2", {}).get("explore_vintage")
    records = explore_data.get("records", [])

    if not records:
        print("Sin más vinos en esta página, paro el bucle.")
        break

    for rec in records:
        vintage = rec.get("vintage", {})
        price_info = rec.get("price") or (rec.get("prices") or [None])[0]
        stats = vintage.get("statistics", {})

        wine_dict = {
            "pagina": page,  # nueva columna
            "vintage_id": vintage.get("id"),
            "nombre": vintage.get("name"),
            "seo_name": vintage.get("seo_name"),
            "year": vintage.get("year"),
            "rating": stats.get("ratings_average"),
            "n_ratings": stats.get("ratings_count"),
            "precio": price_info.get("amount") if price_info else None,
            "moneda": (price_info.get("currency") or {}).get("code") if price_info else None
        }

        all_wines.append(wine_dict)

    time.sleep(1)

# Generar DF y pasarlo a Excel
df_vinos = pd.DataFrame(all_wines)

print(df_vinos.shape)
print(df_vinos.head())

df_vinos.to_excel("vinos_vivino2.xlsx", index=False)
